# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
df = pd.read_csv('city_data.csv')

# Display sample data
df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,port-aux-francais,-49.3500,70.2167,4.11,66,70,11.37,TF,1700026892
1,bethel,41.3712,-73.4140,3.85,79,0,2.06,US,1700026903
2,badger,64.8000,-147.5333,-5.07,80,75,1.54,US,1700026945
3,poronaysk,49.2167,143.1167,-3.55,74,32,0.95,RU,1700026921
4,grytviken,-54.2811,-36.5092,2.59,92,95,1.58,GS,1700026898


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
!pip install folium --upgrade

In [19]:
import folium

# Create a base map centered around the first city's coordinates
base_map = folium.Map(location=[df['Lat'].iloc[0], df['Lng'].iloc[0]], zoom_start=5)

# Add a marker for each city with the size based on humidity
for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 2, 
        popup=row['City'],
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(base_map)

# Display the map
base_map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Assuming city_data_df is your DataFrame with weather data
ideal_weather_df = df[
    (df['Max Temp'] > 21) & (df['Max Temp'] < 27) &
    (df['Wind Speed'] < 4.5) & (df['Cloudiness'] == 0)
]

# Display the DataFrame with ideal weather conditions
ideal_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,gwadar,25.1216,62.3254,25.86,56,0,3.30,PK,1700027087
100,rampur,28.8167,79.0333,25.28,33,0,1.71,IN,1700027206
299,kaka,37.3482,59.6143,24.44,17,0,1.45,TM,1700027241
343,narayangarh,30.4833,77.1333,25.46,36,0,1.96,IN,1700027249
424,kone,-21.0595,164.8658,25.25,55,0,3.60,NC,1700026948
435,al wajh,26.2455,36.4525,22.97,56,0,1.54,SA,1700026958
482,aswan,24.0934,32.9070,22.61,43,0,4.12,EG,1700027003


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:

hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Display the new DataFrame
hotel_df


,City,Country,Lat,Lng,Humidity
44,gwadar,PK,25.1216,62.3254,56
100,rampur,IN,28.8167,79.0333,33
299,kaka,TM,37.3482,59.6143,17
343,narayangarh,IN,30.4833,77.1333,36
424,kone,NC,-21.0595,164.8658,55
435,al wajh,SA,26.2455,36.4525,56
482,aswan,EG,24.0934,32.9070,43


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
import requests

# Geoapify API key (replace with your actual API key)
geoapify_api_key = 'c39a76890a294012871d662f4b30723d'

# find the nearest hotel
def find_nearest_hotel(lat, lng):
    # Geoapify Places API endpoint for finding hotels
    places_url = f'https://api.geoapify.com/v1/places?lat={lat}&lon={lng}&categories=hotels&apiKey={geoapify_api_key}&limit=1'

    # API request
    response = requests.get(places_url)
    data = response.json()

    # Extract hotel information if available
    if 'features' in data and data['features']:
        hotel_name = data['features'][0]['properties']['name']
        country = data['features'][0]['properties']['country']
        return hotel_name, country
    else:
        return None, None

# Iterate over rows in the hotel_df DataFrame and add hotel information
for index, row in hotel_df.iterrows():
    city_lat = row['Lat']
    city_lng = row['Lng']

    # Find the nearest hotel using the Geoapify API
    hotel_name, country = find_nearest_hotel(city_lat, city_lng)

    # Add hotel information to the hotel_df DataFrame
    hotel_df.at[index, 'Hotel Name'] = hotel_name
    hotel_df.at[index, 'Hotel Country'] = country

# Display the updated hotel_df DataFrame
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name,Hotel Country
44,gwadar,PK,25.1216,62.3254,56,NaN,NaN
100,rampur,IN,28.8167,79.0333,33,NaN,NaN
299,kaka,TM,37.3482,59.6143,17,NaN,NaN
343,narayangarh,IN,30.4833,77.1333,36,NaN,NaN
424,kone,NC,-21.0595,164.8658,55,NaN,NaN
435,al wajh,SA,26.2455,36.4525,56,NaN,NaN
482,aswan,EG,24.0934,32.9070,43,NaN,NaN


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.